In [1]:
import unittest
import numpy as np
from collections import deque
from PIL import Image
import pandas as pd
import csv
from skimage.measure import find_contours
from scipy.spatial import distance
from scipy.spatial.distance import euclidean
from skimage.measure import EllipseModel
import os
from pyexpat import features
from re import split
from scipy.spatial import ConvexHull

In [2]:
# Function to calculate the length of a contour
def contour_length(contour):
    return sum(euclidean(contour[i], contour[i+1]) for i in range(len(contour)-1))

def area_polygon(points):
    x = points[:, 0]
    y = points[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def calc_metrics(area_polygon, convex_hull_areas, original_contour_areas, porosities, contour):
    hull = ConvexHull(contour)
    hull_area = area_polygon(contour[hull.vertices])
    contour_area = area_polygon(contour)
    
    convex_hull_areas.append(hull_area)
    original_contour_areas.append(contour_area)
    
    if hull_area == 0:  # Prevent division by zero
        porosity = 0
    else:
        porosity = (hull_area - contour_area) / hull_area
    porosities.append(porosity)

# Function to fit an ellipse to a contour and retrieve its parameters
def fit_ellipse_to_contour(contour):
    ellipse = EllipseModel()
    if ellipse.estimate(contour):  # Check if the ellipse estimation is successful
        xc, yc, a, b, theta = ellipse.params
        return {
            "center_x": xc,
            "center_y": yc,
            "major_axis_length": 2*a,  # Diameter along major axis
            "minor_axis_length": 2*b,  # Diameter along minor axis
            "rotation_angle": np.degrees(theta)  # Convert radians to degrees
        }
    else:
        return "Fit failed"
    

def combine_features(ellipse_params, particle_metrics):
    center_x = []
    center_y = []
    major_axis_length = []
    minor_axis_length = []
    rotation_angle = []

    for element in ellipse_params:
        center_x.append(element['center_x'])
        center_y.append(element['center_y'])
        major_axis_length.append(element['major_axis_length'])
        minor_axis_length.append(element['minor_axis_length'])
        rotation_angle.append(element['rotation_angle'])


    df_temp = pd.DataFrame({'Center X': center_x, 'Center Y': center_y, 'Major Axis Length': major_axis_length,})
    df_new = pd.concat([particle_metrics, df_temp], axis=1)
    return df_new

def load_image(image_path):
    image_path = 'shapes_binary.png'
    image = Image.open(image_path)
    image_np = np.array(image)
    return image_np

def get_features(image_path):
    image_np = load_image(image_path)
    contours = find_contours(image_np, 254)
    contour_lengths = [contour_length(contour) for contour in contours]

    convex_hull_areas = []
    original_contour_areas = []
    porosities = []
    for contour in contours:
        calc_metrics(area_polygon, convex_hull_areas, original_contour_areas, porosities, contour)
    
    metrics = list(zip(convex_hull_areas, original_contour_areas, porosities, contour_lengths))
    particle_metrics = pd.DataFrame(metrics, columns=['Convex Hull Area', 'Contour Area', 'Porosity','Contour Length'])
    ellipse_params = [fit_ellipse_to_contour(contour) for contour in contours]
    features = combine_features(ellipse_params, particle_metrics)
    
    return features


def get_csv_data(shapes_path):
    shapes_df = pd.read_csv(shapes_path)

    shapes_df.columns = ['data']  # Rename the column for easier handling
    parsed_data = shapes_df['data'].str.split(';', expand=True)
    parsed_data.columns = ['x', 'y', 'classification']

    # Convert coordinates to numeric
    parsed_data['y'] = pd.to_numeric(parsed_data['y'])
    parsed_data['x'] = pd.to_numeric(parsed_data['x'])
    return parsed_data
    

# Function to find the closest test particle to each ground truth particle
def find_closest_particles(ground_truth_df, test_df):
    closest_particles = []
    for index, row in ground_truth_df.iterrows():
        ground_truth_coords = (row['x'], row['y'])
        test_coords = test_df[['Center X', 'Center Y']].values
        distances = distance.cdist([ground_truth_coords], test_coords, 'euclidean')
        closest_index = distances.argmin()
        closest_particle = test_df.iloc[closest_index].to_dict()
        closest_particle['ground_truth_classification'] = row['classification']
        closest_particle['ground_truth_x'] = row['x']
        closest_particle['ground_truth_y'] = row['y']
        closest_particles.append(closest_particle)
    
    return pd.DataFrame(closest_particles)

def get_trainingsdata_filepaths(folder_path):
    file_list_csv = []
    file_list_png = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                file_list_csv.append(file_path)
            elif file.endswith(".png"):
                file_path = os.path.join(root, file)
                file_list_png.append(file_path)

    trainings_data_zip = zip(sorted(file_list_csv), sorted(file_list_png))
    trainings_data = [i for i in trainings_data_zip]
    return trainings_data


In [3]:
dict_features = {}
trainings_data = get_trainingsdata_filepaths("training_data")

for csv, png in trainings_data:
    features = get_features(png)
    key = png.split('\\')[-1]
    dict_features[key] = features
    
    parsed_data = get_csv_data(csv)
    # Find the closest particles to the ground truth particles
    matched_particles_df = find_closest_particles(parsed_data, features)
    dict_features[key] = matched_particles_df

In [51]:
dict_features['image_15.png']

,Convex Hull Area,Contour Area,Porosity,Contour Length,Center X,Center Y,Major Axis Length,ground_truth_classification,ground_truth_x,ground_truth_y
0,1201.555113,438.385651,0.635151,174.670423,891.839509,689.102787,37.844150,3-tipped-star,856,819
1,4483.020866,1360.816270,0.696451,508.242745,771.568458,562.026394,54.669344,3-tipped-star,821,590
2,1206.569587,436.909522,0.637891,174.550041,782.021153,349.333592,38.049169,3-tipped-star,736,437
3,4466.537162,1366.563747,0.694044,502.908102,234.951380,444.134517,54.751470,3-tipped-star,261,405
4,1177.722802,433.993287,0.631498,174.049314,477.596741,480.824729,37.704440,3-tipped-star,432,478
5,1310.630931,1287.496268,0.017652,134.339103,280.721685,310.829147,40.680008,3-tipped-star,300,244
6,4472.948178,1371.770209,0.693319,498.965000,136.125297,447.933520,54.489404,3-tipped-star,121,427
7,4484.767400,1356.353283,0.697564,507.128160,49.076022,682.468847,54.848154,3-tipped-star,64,745
8,4501.494539,1369.842422,0.695692,502.792526,778.858653,238.098565,54.910651,3-tipped-star,753,224
9,1195.195621,436.107394,0.635116,175.001706,693.916498,818.242814,38.170234,3-tipped-star,844,853


In [4]:
feature_names = list(dict_features[key].drop(columns= 'ground_truth_classification', axis=1))

class TreeNode():
    def print( self, depth = 0 ):
        raise NotImplementedError()            
    def predict( self, x ):
        raise NotImplementedError()
        
class LeafNode( TreeNode ):
    def __init__( self, output_class, confidence ):
        self.output_class = output_class
        self.confidence = confidence

    def print( self, depth = 0 ):
        print( "  " * depth, "Leaf Node", self.output_class, self.confidence )
    
    def predict( self, x ):
        return self.output_class, self.confidence

class InnerNode( TreeNode ):
    def __init__( self, feature, split_value, left_child, right_child ):
        self.feature = feature
        self.split_value = split_value
        self.left_child = left_child
        self.right_child = right_child

    def print( self, depth = 0 ):
        print( "  " * depth, "Inner Node (", feature_names[self.feature], "<", self.split_value, ")" )
        self.left_child.print( depth + 1 )
        self.right_child.print( depth + 1 )
    
    def predict( self, x ):
        if x[ self.feature ] <= self.split_value:
            return self.left_child.predict( x )
        return self.right_child.predict( x )

In [44]:
def load_in_compatible_format(key):
    X_df = dict_features[key].drop(columns= 'ground_truth_classification', axis=1)
    X = []
    for row in range(X_df.shape[0]):
        t = tuple(X_df.iloc[row])
        X.append(t)

    y = list(dict_features[key]['ground_truth_classification'])

    return X,y

def count_occurance_of_class(data_points, searched_class): #gibt die Anzahl der Vorkommen einer Klasse in einem Datensatz zurück
    count = 0
    for _, class_label in data_points:
        if class_label == searched_class:
            count += 1
    return count

def get_unqiue_classes(data_points): #gibt eine Liste mit den einzigartigen Klassen zurück
    classes = set()
    for _, class_label in data_points:
        classes.add(class_label)
    return list(classes)

def gini_index( data_points ): #berechnet den Gini-Index
    total_count = len(data_points)
    gini = 1 - sum( [ ( count_occurance_of_class(data_points, c) / total_count )**2 for c in get_unqiue_classes(data_points) ] )
    return gini
    

def split_data(data_points, feature_ind, split_value): #teilt die Daten in zwei Teile auf, basierend auf einem Grenzwert(split-value) eines Features
    left = []
    right = []
    for x, _ in data_points:
        if x[feature_ind] <= split_value:
            left.append((x, _))
        else:
            right.append((x, _))
    return left, right

def suggest_split_values( data_points, feature ): #gibt den Median und den Mittelwert der Werte eines Features zurück
    values_of_feature = []
    for x,_ in data_points:
        values_of_feature.append( x[feature] )

    return (np.median(values_of_feature), np.mean(values_of_feature))


#wählt den besten Split aus, basierend auf dem Gini-Index. 
#Es wird der Split mit dem niedrigsten Gini-Index gewählt (also der Split, der die Klassen am besten trennt)
def select_best_split(data_points):
    splits = []
    features = range(len(data_points[0][0]))
    for feature in features:
        split_values = suggest_split_values(data_points, feature)
        for value in split_values:
            left_data, right_data = split_data(data_points, feature, value)
            if len(left_data) == 0 or len(right_data) == 0:
                continue
            left_gini = gini_index(left_data)
            right_gini = gini_index(right_data)
            total_gini = left_gini + right_gini
            splits.append((feature, value, total_gini))
    splits = sorted(splits, key=lambda x: x[2])
    try:
        feature, split_value, _ = splits[0]
    except:
        print("Error: No split found")
    return (feature, split_value)

# Function to check if all points (ignoring labels) have the same value
def all_points_same_ignore_labels(data):
    """
    Returns:
    bool: True if all points are the same, False otherwise.
    """
    points = [tuple(point) for point, _ in data]
    # Return True if the set of points has length 1 or 0, implying all elements are identical or the list is empty
    return len(set(points)) <= 1

def can_stop_recursion(data_points): #überprüft, ob die Rekursion gestoppt werden kann, weil die Datenpunkte homogen sind oder die Anzahl der Datenpunkte zu gering ist
    if len(get_unqiue_classes(data_points)) == 1: #nur eine Klasse vorhanden
        return True
    elif len(data_points) <= 2: #nur 2 oder weniger Datenpunkte vorhanden
        return True
    elif all_points_same_ignore_labels(data_points): #Features sind identisch
        return True
    else: 
        return False

def decision_for_subset(data_points): #gibt die Klasse zurück, die am häufigsten vorkommt und die Konfidenz
    class_labels = get_unqiue_classes(data_points)
    count = [count_occurance_of_class(data_points, c) for c in class_labels]
    max_count = max(count)
    index = count.index(max_count)
    return class_labels[index], max_count / len(data_points)


def build_tree(data_subset):  #rekursive Funktion, die den Baum aufbaut
    if can_stop_recursion(data_subset):
        y, confidence = decision_for_subset(data_subset)
        return LeafNode(y, confidence)
    feature, splitting_value = select_best_split(data_subset)
    left_data, right_data = split_data(data_subset, feature, splitting_value)
    left_child = build_tree(left_data)
    right_child = build_tree(right_data)
    return InnerNode(feature, splitting_value, left_child, right_child)

In [45]:
df = pd.DataFrame()
for val in dict_features.values():
    df = pd.concat([df,val], axis=0)

df = df.sample(frac=1).reset_index(drop=True)
X_df = df.drop(columns= ['ground_truth_classification', 'Center X',	'Center Y','ground_truth_x','ground_truth_y'], axis=1)
X = []
for row in range(X_df.shape[0]):
    t = list(X_df.iloc[row])
    X.append(t)

Y = list(df['ground_truth_classification'])

feature_names = list(dict_features[key].drop(columns= 'ground_truth_classification', axis=1))
data_points = list ( zip( X, Y ) )

tree = build_tree(data_points)
tree.print()

 Inner Node ( Contour Area < 1011.407040737742 )
   Inner Node ( Contour Area < 433.93871321634794 )
     Inner Node ( Center X < 37.94690147481864 )
       Leaf Node 3-tipped-star 0.625
       Inner Node ( Contour Area < 431.88789714872837 )
         Inner Node ( Convex Hull Area < 1187.8976901805959 )
           Leaf Node 3-tipped-star 0.5
           Leaf Node 3-tipped-star 0.5238095238095238
         Leaf Node 8-tipped-star 0.5
     Inner Node ( Contour Length < 174.67042282134196 )
       Inner Node ( Contour Area < 436.9095221199095 )
         Inner Node ( Center X < 37.9331935660025 )
           Leaf Node 3-tipped-star 0.5
           Inner Node ( Convex Hull Area < 1192.9993936350168 )
             Leaf Node 3-tipped-star 0.4444444444444444
             Leaf Node 3-tipped-star 0.46153846153846156
         Leaf Node 3-tipped-star 0.5
       Inner Node ( Porosity < 0.6351163058143064 )
         Inner Node ( Convex Hull Area < 1193.1845120070502 )
           Leaf Node 8-tipped-star 

In [46]:
image_path = 'shapes_binary.png'

features = get_features(image_path)
features_X = features.drop(columns= ['Center X',	'Center Y'], axis=1)

X = []
for row in range(features_X.shape[0]):
    t = tuple(features_X.iloc[row])
    X.append(t)

len(X)

29

In [47]:
for particle in X:
    prediction = tree.predict( particle )
    print(prediction)

('3-tipped-star', 0.5)
('8-tipped-star', 0.5263157894736842)
('8-tipped-star', 1.0)
('3-tipped-star', 0.6875)
('3-tipped-star', 0.5)
('3-tipped-star', 0.5)
('3-tipped-star', 0.45454545454545453)
('3-tipped-star', 0.5238095238095238)
('8-tipped-star', 0.6)
('3-tipped-star', 0.36363636363636365)
('3-tipped-star', 0.4)
('8-tipped-star', 0.375)
('3-tipped-star', 0.38461538461538464)
('3-tipped-star', 0.5)
('3-tipped-star', 0.4375)
('3-tipped-star', 0.5)
('3-tipped-star', 0.5)
('8-tipped-star', 0.5555555555555556)
('3-tipped-star', 0.4117647058823529)
('8-tipped-star', 0.4666666666666667)
('3-tipped-star', 0.4444444444444444)
('3-tipped-star', 0.5882352941176471)
('3-tipped-star', 0.46153846153846156)
('3-tipped-star', 0.625)
('3-tipped-star', 0.5)
('3-tipped-star', 0.4444444444444444)
('8-tipped-star', 0.5)
('3-tipped-star', 0.5454545454545454)
('3-tipped-star', 0.625)
